In [31]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from xgboost import XGBRFClassifier
import numpy as np
import pandas as pd

In [32]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=="Barclays Premier League") & (df.season >= 2019)]

In [33]:
df['h_margin'] = df['score1'] - df['score2']

In [34]:
results = []

In [35]:
for x in df['h_margin']:
    if x > 0:
        results.append("HOME WIN")
    elif x == 0:
        results.append("DRAW")
    else:
        results.append("AWAY WIN")

In [36]:
df['result'] = results

In [37]:
df['xg_h_margin'] = df['xg1'] - df['xg2']

In [38]:
xg_results = []

In [39]:
for y in df['xg_h_margin']:
    if y > 0:
        xg_results.append("HOME xG WIN")
    elif y == 0:
        xg_results.append("xG DRAW")
    else:
        xg_results.append("AWAY xG WIN")

In [40]:
df['xg_result'] = xg_results

In [41]:
df = df.drop(columns=['h_margin', 'xg_h_margin'])

In [47]:
comp_df = df[['result', 'xg_result']]
comp_ran = range(len(comp_df))

In [55]:
comparisons = []

In [56]:
comp_df.iloc[1,:][0]

'AWAY WIN'

In [57]:
for z in comp_ran:
    z = comp_df.iloc[z,:]
    if (z[0] == "HOME WIN") & (z[1]=="HOME xG WIN"):
        comparisons.append("HOME EXPECTED")
    elif (z[0] == "AWAY WIN") & (z[1]=="HOME xG WIN"):
        comparisons.append("AWAY UNEXPECTED")
    elif (z[0] == "HOME WIN") & (z[1]=="AWAY xG WIN"):
        comparisons.append("HOME UNEXPECTED")
    elif (z[0] == "AWAY WIN") & (z[1]=="AWAY xG WIN"):
        comparisons.append("AWAY EXPECTED")
    else:
        comparisons.append("DRAW")

In [59]:
df['comparison'] = comparisons

In [60]:
X = df[['score1','score2','xg1','xg2']]
y = df['comparison']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=6)

In [79]:
model = XGBRFClassifier(max_depth=6, eval_metric='mlogloss').fit(X_train, y_train)

In [80]:
kfold = KFold(n_splits=10, shuffle=True)

In [85]:
import warnings
warnings.filterwarnings("ignore")
cv_scores = cross_val_score(model, X_train, y_train, cv=kfold)

In [86]:
print("Cross Val Scores:", cv_scores)

Cross Val Scores: [0.96491228 0.87719298 0.96491228 0.9122807  0.87719298 0.92982456
 0.94736842 0.94642857 0.91071429 0.96428571]


In [87]:
predictions = model.predict(X_test)

In [94]:
accuracy = accuracy_score(y_test, predictions) * 100

In [95]:
print("Test Accuracy: {}%".format(np.round(accuracy, 3)))

Test Accuracy: 92.593%
